## Read Khan Academy Subject Page

In [1]:
import os
from bs4 import BeautifulSoup
import requests
import pymongo

### Read Subject Page

In [2]:
# Read Khan Academy Subject Page
headers = {
   'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0',
   'X-Requested-With': 'XMLHttpRequest'
}
domain = 'https://www.khanacademy.org'
# source = domain + '/math/statistics-probability'
source = domain + '/math/probability'
response = requests.get(source, headers=headers)
response

<Response [200]>

In [7]:
print(dir(response))

['__attrs__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_content', '_content_consumed', '_next', 'apparent_encoding', 'close', 'connection', 'content', 'cookies', 'elapsed', 'encoding', 'headers', 'history', 'is_permanent_redirect', 'is_redirect', 'iter_content', 'iter_lines', 'json', 'links', 'next', 'ok', 'raise_for_status', 'raw', 'reason', 'request', 'status_code', 'text', 'url']


#### Read units

In [3]:
# Mount lessons array

# Lessons
lessons = []

# Get Page headings
unit_sequence = 0
# bsSubject = BeautifulSoup(response.text, 'lxml')
bsSubject = BeautifulSoup(response.text, 'html.parser')

field   = bsSubject.find('div', attrs={"aria-label": "breadcrumbs"}).get_text()
subject = bsSubject.find('h1',  attrs={"data-test-id": "unit-block-title"}).get_text()

# print(field)

for unit_block in bsSubject.findAll('div', class_='_12yy6f6l'):
   unit_sequence += 1
   unit_title = unit_block.find('h3').get_text()
   print(unit_title)

   for group in unit_block.findAll('a', attrs={"data-test-id": "lesson-link"}):
      group_title = group.get('title')
      group_link  = group.get('href')
      
      # Load group page
      group_page = requests.get(domain + group_link, headers=headers)
      print("\t" + group_title)
      bsGroup = BeautifulSoup(group_page.text, 'html.parser')
      unit_group = bsGroup.find('ul', attrs={"data-test-id": "learnable-content-cells"})

      # Browse through lessons
      for lesson in unit_group.findAll('li'):
         lesson_title = lesson.get_text()
         lesson_link = lesson.find('a').get('href')

         # Get lesson type
         lesson_type = lesson.find(['span', 'svg'], attrs={"role": "img"}).get('aria-label')
         if lesson_type not in ['Article', 'Video']:
            lesson_type = 'Exercise'

         # print("\t\t" + lesson_title)
         print("\t\t" + lesson_type)
         # print("\t\t" + lesson_link)

         # Load lesson metadados
         lesson_page = requests.get(domain + lesson_link, headers=headers)
         bsLesson = BeautifulSoup(lesson_page.text, 'html.parser')



Unit 1: Displaying a single quantitative variable
	Frequency tables and dot plots
		Video
		Exercise
		Exercise
	Histograms
		Video
		Exercise
		Video
		Exercise
	Mean and median in data displays
		Video
		Video
		Exercise
		Video
		Exercise
		Article
	Interquartile range
		Video
		Exercise
	Box and whisker plots
		Video
		Video
		Exercise
		Video


KeyboardInterrupt: 

#### Read groups of exercises and save them in the database

In [5]:
# Read group of exercises

# Group of exercises
groups = []

# Get articles tab
group_sequence = 0
bs = BeautifulSoup(response.text, 'lxml')
for article in bs.findAll('div', {'class':'article-tab'}):

   for item in article.children:

      if str(item)[:3] == '<p>':    # Category titles

         group_item = {}
         category = item.text[:-2].strip()
         group_sequence += 100  

      elif str(item)[:3] == '<ul':  # Group items

         bs2 = BeautifulSoup(str(item), 'lxml')
         sequence = group_sequence

         for ex_page in bs2.findAll('a'):

            # Group properties
            raw_name = ex_page.text.split(' [ ')
            if len(raw_name) == 2:

               _id = raw_name[0]
               sequence += 1
               url = ex_page['href']
               url = url if url[:4] == 'http' else domain + url


               if _id != 'Python Challenges: Part -1':
                  quantity = int(''.join(c for c in raw_name[1] if c.isnumeric()))
               else:
                  quantity = 64

               group_item = {
                  '_id':      _id,
                  'category': category,
                  'sequence': sequence,
                  'url':      url,
                  'quantity': quantity,
                  'qtd_done': 0,
                  'header': ''
               }
               groups.append(group_item)

In [6]:
# print out an array or dictionary nicely
import json
print(json.dumps(groups, indent=4))

[
    {
        "_id": "Python Basic (Part -I)",
        "category": "List of Python Exercises",
        "sequence": 101,
        "url": "https://www.w3resource.com/python-exercises/python-basic-exercises.php",
        "quantity": 150,
        "qtd_done": 0,
        "header": ""
    },
    {
        "_id": "Python Basic (Part -II)",
        "category": "List of Python Exercises",
        "sequence": 102,
        "url": "https://www.w3resource.com/python-exercises/basic/",
        "quantity": 145,
        "qtd_done": 0,
        "header": ""
    },
    {
        "_id": "Python Programming Puzzles",
        "category": "List of Python Exercises",
        "sequence": 103,
        "url": "https://www.w3resource.com/python-exercises/puzzles/index.php",
        "quantity": 100,
        "qtd_done": 0,
        "header": ""
    },
    {
        "_id": "Python built-in Modules",
        "category": "List of Python Exercises",
        "sequence": 104,
        "url": "https://www.w3resource.com/pyt

In [ ]:
# Save group of the exercises in the database

# Create w3resource python database
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['khan-academy']

# group of exercises collection
colLessons = db["lessons"]

x = colLessons.delete_many({})
x = colLessons.insert_many(lessons)

#print list of the _id values of the inserted documents:
print(x.inserted_ids)